## Тук ще пробваме невронни мрежи, за да се справим с токсичните коментари

Намерих един kernel в Kaggle, където използват Bidirection LSTM, за да получат доста добри резултати.

За embeddings се ползва GloVe.

## Защо LSTM?

LSTM невронните мрежи са известни с това, че се справят доста добре в класификацията на текст.

## Защо Bidirectional?

Когато се използва BLSTM, в началото на input layer-a се тренират 2 LSTM мрежи вместо една - първата се тренира върху текста, а втората върху текста обърнат наобратно. Експерименти през последните години показват, че това увеличава значително performance-a на модели, които целят sequential classification. Ключовата прична за това е, че чрез тях мрежата знае за миналото и за бъдещето по едно и също време.

In [1]:
import numpy as np
import pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


Взимаме си данните първо, както направихме и в другия файл.

In [2]:
train_data = pd.read_csv('./data/train.csv.zip', index_col=['id'])
test_data = pd.read_csv('./data/test.csv.zip', index_col=['id'])

list_sentences_train = train_data["comment_text"].fillna("empty").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train_data[list_classes].values
list_sentences_test = test_data["comment_text"].fillna("empty").values

Някои базови променливи за embedding-ите.

In [3]:
EMEDDING_SIZE = 50 # how big is each word vector
MAX_FEATURES = 20000 # how many unique words to use (i.e num rows in embedding vector)
MAX_WORDS = 100 # max number of words in a comment to use

Стандартен начин на preprocessing, който се препоръчва от keras. Превръщаме коментарите в списък от индекси.

In [4]:
tokenizer = Tokenizer(num_words=MAX_FEATURES)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_train = pad_sequences(list_tokenized_train, maxlen=MAX_WORDS)
X_test = pad_sequences(list_tokenized_test, maxlen=MAX_WORDS)

Ще напълним един речник, където пазим word -> vector репрезентациите.

In [5]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')

with open('./data/glove/glove.6B.50d.txt') as f:
    glove = f.readlines()
    print(glove[0])
    embeddings_index= dict(get_coefs(*l.strip().split()) for l in glove)
    
print(embeddings_index['the'])

the 0.418 0.24968 -0.41242 0.1217 0.34527 -0.044457 -0.49688 -0.17862 -0.00066023 -0.6566 0.27843 -0.14767 -0.55677 0.14658 -0.0095095 0.011658 0.10204 -0.12792 -0.8443 -0.12181 -0.016801 -0.33279 -0.1552 -0.23131 -0.19181 -1.8823 -0.76746 0.099051 -0.42125 -0.19526 4.0071 -0.18594 -0.52287 -0.31681 0.00059213 0.0074449 0.17778 -0.15897 0.012041 -0.054223 -0.29871 -0.15749 -0.34758 -0.045637 -0.44251 0.18785 0.0027849 -0.18411 -0.11514 -0.78581

[ 4.1800e-01  2.4968e-01 -4.1242e-01  1.2170e-01  3.4527e-01 -4.4457e-02
 -4.9688e-01 -1.7862e-01 -6.6023e-04 -6.5660e-01  2.7843e-01 -1.4767e-01
 -5.5677e-01  1.4658e-01 -9.5095e-03  1.1658e-02  1.0204e-01 -1.2792e-01
 -8.4430e-01 -1.2181e-01 -1.6801e-02 -3.3279e-01 -1.5520e-01 -2.3131e-01
 -1.9181e-01 -1.8823e+00 -7.6746e-01  9.9051e-02 -4.2125e-01 -1.9526e-01
  4.0071e+00 -1.8594e-01 -5.2287e-01 -3.1681e-01  5.9213e-04  7.4449e-03
  1.7778e-01 -1.5897e-01  1.2041e-02 -5.4223e-02 -2.9871e-01 -1.5749e-01
 -3.4758e-01 -4.5637e-02 -4.4251e-01  1

Сега ще използваме речника, за да си направим embeddings матрицата. За думите, които нямаме, ще използваме random стойности.

In [6]:
all_embs = np.stack(embeddings_index.values())
emb_mean, emb_std = all_embs.mean(), all_embs.std()
emb_mean, emb_std

(0.020940498, 0.6441043)

In [7]:
word_index = tokenizer.word_index
nb_words = min(MAX_FEATURES, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, EMEDDING_SIZE))

for word, i in word_index.items():
    if i >= MAX_FEATURES:
        continue
    
    embedding_vector = embeddings_index.get(word)
    
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

### Преди да започнем

Тук е времето да кажа, че е много важно да си четеш лекциите **внимателно**, за да не пускаш моделите си за 3ти път!!! Вчера тренирах със `sigmoid`, което е напълно грешно за моята задача, въпреки че постигах доста високи резултати. Тук е правилно да се използва `softmax`, тъй като моята задача е `multiclasss classification`. Със `sigmoid` имах успех от около 98.3. Сега ще видим какво ще излезе, очаквам по-слаб резултат.

In [26]:
inp = Input(shape=(MAX_WORDS,))
x = Embedding(MAX_FEATURES, EMEDDING_SIZE, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="softmax")(x)

blstm = Model(inputs=inp, outputs=x)
blstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

blstm.fit(X_train, y, batch_size=32, epochs=2, validation_split=0.1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143613/143613 [==============================] - 630s 4ms/step - loss: 0.2220 - acc: 0.9642 - val_loss: 0.2193 - val_acc: 0.9634
Epoch 2/2
143613/143613 [==============================] - 623s 4ms/step - loss: 0.2181 - acc: 0.9648 - val_loss: 0.2184 - val_acc: 0.9649


### Супер!

Току-що направих първата си невронна мрежа с помощта на keras.


### Сега ще пробвам малко модификации

На една от последните лекции казахте, че GRU се справят добре, колкото LSTM, но с доста по-малко изчисления. Ще пробвам да използвам GRU вместо LSTM. Ще сложим още едно ниво с GRU, този път unidirectional. Слагам и Dropout, за да избягаме от overfitting. Ще увелича и броя на епохите.

In [27]:
from keras.layers import GRU

inp = Input(shape=(MAX_WORDS,))
x = Embedding(MAX_FEATURES, EMEDDING_SIZE, weights=[embedding_matrix])(inp)
x = Bidirectional(GRU(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = Dropout(0.1)(x)
x = GRU(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1)(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="softmax")(x)

gru = Model(inputs=inp, outputs=x)
gru.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

gru.fit(X_train, y, batch_size=32, epochs=3, validation_split=0.1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/3
143613/143613 [==============================] - 791s 6ms/step - loss: 0.2221 - acc: 0.9641 - val_loss: 0.2198 - val_acc: 0.9633
Epoch 2/3
143613/143613 [==============================] - 759s 5ms/step - loss: 0.2184 - acc: 0.9648 - val_loss: 0.2187 - val_acc: 0.9647
Epoch 3/3
143613/143613 [==============================] - 762s 5ms/step - loss: 0.2169 - acc: 0.9655 - val_loss: 0.2183 - val_acc: 0.9645


Както и предположих, GRU се справи добре както и LSTM-a. За 2 епохи постигна същия резултат. Наблюдава се подобрение при повече епохи.

### Следващ вариант

Следващата вариация, която ще използваме е да добавим конволюционна невронна мрежа след BLSTM-a. В [този paper](http://www.aclweb.org/anthology/C16-1329) се разказва за добри резултати при използването на тази комбинация за sequence classification. Ще пробвам и с малко повече епохи.

In [9]:
from keras.layers import Conv1D, MaxPooling1D, Flatten

inp = Input(shape=(MAX_WORDS,))
x = Embedding(MAX_FEATURES, EMEDDING_SIZE, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(300, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = Conv1D(100, 3, activation='relu', padding='same')(x)
x = Conv1D(100, 3, activation='relu')(x)
x = MaxPooling1D()(x)
x = Dropout(0.2)(x)
x = Flatten()(x)
x = Dense(300, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(200, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(100, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)

model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y, batch_size=32, epochs=4, validation_split=0.1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/4
143613/143613 [==============================] - 3866s 27ms/step - loss: 0.0642 - acc: 0.9785 - val_loss: 0.0519 - val_acc: 0.9812
Epoch 2/4
143613/143613 [==============================] - 3862s 27ms/step - loss: 0.0513 - acc: 0.9817 - val_loss: 0.0529 - val_acc: 0.9810
Epoch 3/4
143613/143613 [==============================] - 3848s 27ms/step - loss: 0.0463 - acc: 0.9827 - val_loss: 0.0527 - val_acc: 0.9820
Epoch 4/4
143613/143613 [==============================] - 3845s 27ms/step - loss: 0.0428 - acc: 0.9836 - val_loss: 0.0513 - val_acc: 0.9824


В [примерите на keras](https://github.com/keras-team/keras/tree/ce4947cbaf380589a63def4cc6eb3e460c41254f/examples) намерих един, който прави класификация на текстове от IMDB. Вътре използват Conv1D + LSTM, като мен, но лагат конволюционната мрежа преди рекурентната. Потърсих и други примери  в интернет и там правят същото. Смятам, че горната мрежа не се справи много добре, защото съм разменил местата на мрежите.

Ще опитаме да имплементираме [примера на keras](https://github.com/keras-team/keras/blob/ce4947cbaf380589a63def4cc6eb3e460c41254f/examples/imdb_cnn_lstm.py), като този път разположа мрежите по правилния начин. Те не ползват BLSTM, а просто LSTM, но аз ще се придържам към bidirectional варианта.

Там имат само по 25 000 реда в сравнение с нашите ~160 000. Пише, че заради това стигат само 2 епохи. Аз ще пробвам с повече, заради по-големия обем данни.

In [31]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Activation

FILTERS = 64
KERNEL_SIZE = 5
POOL_SIZE = 4

conv_blstm = Sequential()
conv_blstm.add(Embedding(MAX_FEATURES, EMEDDING_SIZE, input_length=MAX_WORDS))
conv_blstm.add(Dropout(0.1))

conv_blstm.add(Conv1D(FILTERS,
                 KERNEL_SIZE,
                 padding='valid',
                 activation='relu',
                 strides=1))
conv_blstm.add(MaxPooling1D(pool_size=POOL_SIZE))

conv_blstm.add(Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1)))
conv_blstm.add(GlobalMaxPool1D())

conv_blstm.add(Dense(50))
conv_blstm.add(Activation('relu'))
conv_blstm.add(Dropout(0.1))
conv_blstm.add(Dense(6))
conv_blstm.add(Activation('softmax'))

conv_blstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

conv_blstm.fit(X_train, y, batch_size=32, epochs=5, validation_split=0.1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/5
143613/143613 [==============================] - 293s 2ms/step - loss: 0.2228 - acc: 0.9641 - val_loss: 0.2205 - val_acc: 0.9637
Epoch 2/5
143613/143613 [==============================] - 279s 2ms/step - loss: 0.2187 - acc: 0.9651 - val_loss: 0.2197 - val_acc: 0.9659
Epoch 3/5
143613/143613 [==============================] - 281s 2ms/step - loss: 0.2165 - acc: 0.9661 - val_loss: 0.2196 - val_acc: 0.9637
Epoch 4/5
143613/143613 [==============================] - 283s 2ms/step - loss: 0.2144 - acc: 0.9669 - val_loss: 0.2208 - val_acc: 0.9642
Epoch 5/5
143613/143613 [==============================] - 284s 2ms/step - loss: 0.2126 - acc: 0.9678 - val_loss: 0.2212 - val_acc: 0.9633


Интересно дали с увеличаване на епохите, моделът ще се стабилизира.

In [32]:
conv_blstm.fit(X_train, y, batch_size=32, epochs=10, validation_split=0.1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/10
143613/143613 [==============================] - 281s 2ms/step - loss: 0.2108 - acc: 0.9684 - val_loss: 0.2231 - val_acc: 0.9632
Epoch 2/10
143613/143613 [==============================] - 283s 2ms/step - loss: 0.2094 - acc: 0.9690 - val_loss: 0.2258 - val_acc: 0.9633
Epoch 3/10
143613/143613 [==============================] - 283s 2ms/step - loss: 0.2082 - acc: 0.9695 - val_loss: 0.2257 - val_acc: 0.9618
Epoch 4/10
143613/143613 [==============================] - 281s 2ms/step - loss: 0.2076 - acc: 0.9696 - val_loss: 0.2274 - val_acc: 0.9623
Epoch 5/10
143613/143613 [==============================] - 282s 2ms/step - loss: 0.2068 - acc: 0.9700 - val_loss: 0.2267 - val_acc: 0.9628
Epoch 6/10
143613/143613 [==============================] - 284s 2ms/step - loss: 0.2063 - acc: 0.9702 - val_loss: 0.2322 - val_acc: 0.9624
Epoch 7/10
143613/143613 [==============================] - 283s 2ms/step - loss: 0.2057 - acc: 0.9704 - val_

Постигнахме по-висока оценка, но за сметка на това validation accuracy-то ни падна. Това означава, че моделът ни по-скоро overfit-ва.

Да видим как ще се справят моделите на тренировъчните данни.

In [36]:
print('Bidirectional LSTM')
y_test_blstm = blstm.predict([X_test], batch_size=1024, verbose=1)
print('Bidirectioanl GRU')
y_test_gru = gru.predict([X_test], batch_size=1024, verbose=1)
print('Convlution + BLSTM')
y_test_conv_blstm = conv_blstm.predict([X_test], batch_size=1024, verbose=1)

Bidirectional LSTM
153164/153164 [==============================] - 85s 553us/step
Bidirectioanl GRU
153164/153164 [==============================] - 125s 816us/step
Convlution + BLSTM
153164/153164 [==============================] - 48s 313us/step


In [39]:
def make_sample(y, suff):
    sample_submission = pd.read_csv('./data/sample_submission.csv.zip')
    sample_submission[list_classes] = y
    sample_submission.to_csv(f'./submissions/submission_{suff}.csv', index=False)

make_sample(y_test_blstm, 'blstm')
make_sample(y_test_gru, 'gru')
make_sample(y_test_conv_blstm, 'conv_blstm')